# Agent Scratch Book

In [1]:
import logging
import torch
from dotenv import load_dotenv

from aicraft.models import GPT, LLaMa, Qwen, DeepSeek, Gemma
from aicraft.types import InferenceRequest

load_dotenv()

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
device

device(type='mps')

## Initialise Models

In [2]:
gemma2 = Gemma() # obliterated
qwen2 = Qwen() # For faster generations
llama3 = LLaMa() # More capable, but slower
deepseek2 = DeepSeek() # Code generator
gpt4o = GPT() # Fast, Very capable, but costly

In [3]:
from pydantic import BaseModel


class SQLResponse(BaseModel):
    """
    Data model for SQL response
    contentType: str one of [application/json, text/plain, text/markdown]
    sql: str
    """
    contentType: str
    sql: str
    
logger.info(SQLResponse.__doc__)

INFO:__main__:
    Data model for SQL response
    contentType: str one of [application/json, text/plain, text/markdown]
    sql: str
    


In [4]:
SYSTEM_PROMPT = """
Instruction:
You are postgres database developer who is instructed to write SQL queries to fetch data from the database. Your SQL is required to be efficient and not contain any syntax errors. Here are the DDLs for the database:

DDL:
create table behaviour.interest (
  gender text, -- User gender: Male | Female | Non-Binary
  age_range text, -- User age range: 13-17 | 18-24 | 25-34 | 35-44 | 45-54 | 55-64 |65+ 
  city_name text, -- User city
  state_name text, -- User state
  country_name text, -- User country
  user_count integer, -- Number of users that endorsed
  subject text, -- Product, service, or content endorsed
  description text, -- Description of product, service, or content
  link text, -- Link to product, service, or content
  image_link text, -- Link to image of product, service, or content
  tag_1 text, -- User provided category for product, service or content
  tag_2 text,
  tag_3 text,
  tag_4 text,
  tag_5 text,
  tag_6 text,
  tag_7 text,
  tag_8 text,
  tag_9 text, 
  tag_10 text
);

Respond back only with error free sql syntax only
"""
QUESTION = "Respond back with SQL only: which age group have provided the highest number of endorsements across the entire dataset?"
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": QUESTION },
]

single_message = [{"role": "user", "content":messages[0]["content"] + messages[1]["content"] }]

In [5]:
%%time
response = gpt4o.generate(InferenceRequest(
    messages=messages,
    response_format=SQLResponse,
))
response

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


CPU times: user 17.1 ms, sys: 94.7 ms, total: 112 ms
Wall time: 2.04 s


InferenceResponse(role='assistant', content=SQLResponse(contentType='application/json', sql='SELECT age_range, SUM(user_count) AS total_endorsements\nFROM behaviour.interest\nGROUP BY age_range\nORDER BY total_endorsements DESC\nLIMIT 1;'))

In [6]:
%%time
response = deepseek2.generate(InferenceRequest(
    messages=messages,
    response_format=SQLResponse,
))
response

char ::= [^"\] | [\] char_1 
char_1 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
contentType-kv ::= ["] [c] [o] [n] [t] [e] [n] [t] [T] [y] [p] [e] ["] space [:] space string 
space ::= space_7 
string ::= ["] string_8 ["] space 
root ::= [{] space contentType-kv [,] space sql-kv [}] space 
sql-kv ::= ["] [s] [q] [l] ["] space [:] space string 
space_7 ::= [ ] | 
string_8 ::= char string_8 | 
CPU times: user 510 ms, sys: 662 ms, total: 1.17 s
Wall time: 9.72 s


InferenceResponse(role='assistant', content=SQLResponse(contentType='text', sql='SELECT age_range, SUM(user_count) AS total_endorsements FROM behaviour.interest GROUP BY age_range ORDER BY total_endorsements DESC LIMIT 1'))

In [7]:
%%time
response = llama3.generate(InferenceRequest(
    messages=messages,
    response_format=SQLResponse,
))
response

char ::= [^"\] | [\] char_1 
char_1 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
contentType-kv ::= ["] [c] [o] [n] [t] [e] [n] [t] [T] [y] [p] [e] ["] space [:] space string 
space ::= space_7 
string ::= ["] string_8 ["] space 
root ::= [{] space contentType-kv [,] space sql-kv [}] space 
sql-kv ::= ["] [s] [q] [l] ["] space [:] space string 
space_7 ::= [ ] | 
string_8 ::= char string_8 | 
CPU times: user 345 ms, sys: 465 ms, total: 809 ms
Wall time: 2.69 s


InferenceResponse(role='assistant', content=SQLResponse(contentType='text/plain', sql='SELECT age_range, user_count FROM behaviour.interest ORDER BY user_count DESC LIMIT 1;'))

In [8]:
%%time
response = qwen2.generate(InferenceRequest(
    messages=messages,
    response_format=SQLResponse,
))
response

char ::= [^"\] | [\] char_1 
char_1 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
contentType-kv ::= ["] [c] [o] [n] [t] [e] [n] [t] [T] [y] [p] [e] ["] space [:] space string 
space ::= space_7 
string ::= ["] string_8 ["] space 
root ::= [{] space contentType-kv [,] space sql-kv [}] space 
sql-kv ::= ["] [s] [q] [l] ["] space [:] space string 
space_7 ::= [ ] | 
string_8 ::= char string_8 | 
CPU times: user 511 ms, sys: 305 ms, total: 817 ms
Wall time: 1.45 s


InferenceResponse(role='assistant', content=SQLResponse(contentType='sql', sql='SELECT gender, age_range, MAX(user_count) FROM behavior.interest GROUP BY gender, age_range ORDER BY MAX(user_count) DESC LIMIT 1'))

In [9]:
%%time
response = gemma2.generate(InferenceRequest(
    messages=messages,
    response_format=SQLResponse,
))
response

char ::= [^"\] | [\] char_1 
char_1 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
contentType-kv ::= ["] [c] [o] [n] [t] [e] [n] [t] [T] [y] [p] [e] ["] space [:] space string 
space ::= space_7 
string ::= ["] string_8 ["] space 
root ::= [{] space contentType-kv [,] space sql-kv [}] space 
sql-kv ::= ["] [s] [q] [l] ["] space [:] space string 
space_7 ::= [ ] | 
string_8 ::= char string_8 | 
CPU times: user 871 ms, sys: 538 ms, total: 1.41 s
Wall time: 2.5 s


InferenceResponse(role='assistant', content=SQLResponse(contentType='text/sql', sql='SELECT age_range, COUNT(*) AS endorsement_count FROM behaviour.interest GROUP BY age_range ORDER BY endorsement_count DESC LIMIT 1'))